# Preprocessing CONUS404 and reference data

Short paragraph describing what is about to happen

<details>
  <summary>Guide to pre-requisites and learning outcomes...&lt;click to expand&gt;</summary>
  
  <table>
    <tr>
      <td>Pre-Requisites
      <td>To get the most out of this notebook, you should already have an understanding of these topics: 
        <ul>
        <li>pre-req one
        <li>pre-req two
        </ul>
    <tr>
      <td>Expected Results
      <td>At the end of this notebook, you should be able to: 
        <ul>
        <li>outcome one
        <li>outcome two
        </ul>
  </table>
</details>

In [16]:
# library imports
import fsspec
import hvplot.xarray
import intake
import os
import warnings
import requests
import rioxarray
import dask
import metpy

from shapely.geometry import Polygon
from dask.distributed import LocalCluster, Client
from pygeohydro import pygeohydro
from fsspec.implementations.ftp import FTPFileSystem

import xarray as xr
import geopandas as gpd
import pandas as pd
import geoviews as gv
import dask.dataframe as dd
import cartopy.crs as ccrs #testing
import numpy as np #testing
# add polyedit, polydraw
warnings.filterwarnings('ignore')

## Retrieving data from HPC or the Cloud
#### The process varies based on where the notebook is being run but generally looks this:
1. (Done already) Connect to workspace (local, HPC, or QHUB) and open notebook
2. Start Dask client 
3. Pull in data from source
4. Process the data into usable file format, size, and extent

# Update tp helper function after repo consolidation
## **Start a Dask client using an appropriate Dask Cluster** 
This is an optional step, but can speed up data loading significantly, especially when accessing data from the Cloud.

In [2]:
def configure_cluster(machine):
    ''' Helper function to configure cluster
    '''
    if machine == 'denali':
        from dask.distributed import LocalCluster, Client
        cluster = LocalCluster(threads_per_worker=1)
        client = Client(cluster)
    
    elif machine == 'tallgrass':
        from dask.distributed import Client
        from dask_jobqueue import SLURMCluster
        cluster = SLURMCluster(queue='cpu', cores=1, interface='ib0',
                               job_extra=['--nodes=1', '--ntasks-per-node=1', '--cpus-per-task=1'],
                               memory='6GB')
        cluster.adapt(maximum_jobs=30)
        client = Client(cluster)
        
    elif machine == 'local':
        import os
        import warnings
        from dask.distributed import LocalCluster, Client
        warnings.warn("Running locally can result in costly data transfers!\n")
        n_cores = os.cpu_count() # set to match your machine
        cluster = LocalCluster(threads_per_worker=n_cores)
        client = Client(cluster)
        
    elif machine in ['esip-qhub-gateway-v0.4']:   
        import sys, os
        sys.path.append(os.path.join(os.environ['HOME'],'shared','users','lib'))
        import ebdpy as ebd
        aws_profile = 'esip-qhub'
        ebd.set_credentials(profile=aws_profile)

        aws_region = 'us-west-2'
        endpoint = f's3.{aws_region}.amazonaws.com'
        ebd.set_credentials(profile=aws_profile, region=aws_region, endpoint=endpoint)
        worker_max = 30
        client,cluster = ebd.start_dask_cluster(profile=aws_profile, worker_max=worker_max, 
                                              region=aws_region, use_existing_cluster=True,
                                              adaptive_scaling=False, wait_for_cluster=False, 
                                              worker_profile='Medium Worker', propagate_env=True)
        
    return client, cluster

### Select workspace from the following list:
'denali', 'tallgrass', 'local', or 'esip-qhub-gateway-v0.4'

### View available datasets from the Intake Catalog and choose which to use
Note: Select datasets that end in "onprem" if running on Denali/Tallgrass HPC or cloud data if working on QHub or local.

In [3]:
url = 'https://raw.githubusercontent.com/hytest-org/hytest/main/dataset_catalog/hytest_intake_catalog.yml'
cat = intake.open_catalog(url)
list(cat)

['conus404-hourly-onprem',
 'conus404-hourly-cloud',
 'conus404-daily-onprem',
 'conus404-daily-cloud',
 'nwis-streamflow-usgs-gages-onprem',
 'nwis-streamflow-usgs-gages-cloud',
 'nwm21-streamflow-usgs-gages-onprem',
 'nwm21-streamflow-usgs-gages-cloud',
 'nwm21-streamflow-cloud',
 'nwm21-scores',
 'lcmap-cloud',
 'conus404-hourly-cloud-dev']

### You can setup your client and dataset on QHub like this:
Uncomment single commented spaces (#) to run

In [4]:
# set machine
machine = 'esip-qhub-gateway-v0.4'

# use configure cluster helper function to setup dask
client, cluster = configure_cluster(machine)

# set dataset
dataset = 'conus404-hourly-cloud'

Region: us-west-2
Existing Dask clusters:
Cluster Index c_idx: 0 / Name: dev.cbbdc097b10643ab977ef43a6f4a48f1 ClusterStatus.RUNNING
Using existing cluster [0].
Setting Fixed Scaling workers=30
Reconnect client to clear cache
client.dashboard_link (for new browser tab/window or dashboard searchbar in Jupyterhub):
https://jupyter.qhub.esipfed.org/gateway/clusters/dev.cbbdc097b10643ab977ef43a6f4a48f1/status
Propagating environment variables to workers
Using environment: users/pangeo


### Or if on the HPC you can setup your client and dataset like this:
Uncomment single commented spaces (#) to run

In [ ]:
## set machine
# machine = os.environ['SLURM_CLUSTER_NAME']

## use configure_cluster helper function to setup dask
# client, cluster = configure_cluster(machine)

## set dataset
# dataset = 'conus404-hourly-onprem'

## **Retrieve CONUS404 from source and tranform it to a Dask array**

In [ ]:
# double check that dataset is in catalog (cat)
dataset = 'conus404-hourly-cloud'
cat[dataset]

Transform data to dask array

In [ ]:
ds = cat[dataset].to_dask()

View dataset metadata. To view variables, expand the "Data variables" section. 

For this tutorial, we will be working with accumulated precipitation (PREC_ACC_NC), air temperature (TK), and surface net radiation (RNET) variables. RNET req 

In [ ]:
ds

## **Explore the variable** 
(sometimes called exploratory data analysis (EDA) or exploratory spatial data analysis (ESDA) when it contains cartographic data)

### Lets look at the accumulated precipitation variable by first subsetting the larger dataset. 
Notice the information in the array and chunk columns as well as the coordinates (in particular *time*) and the units.

In [ ]:
# variable PREC_ACC_NC
prec = ds.PREC_ACC_NC
prec

### Next, lets visualize a map of the data at specific time step.

In [ ]:
prec_time = prec.sel(time='2014-03-01 00:00').load()

In the previous cell, the .sel() method filters the dataset by the *time* coordinate through "time=" and then uses the .load() method to load the dataset into memory.

Now, let's visualize the dataset using the [QuadMesh](https://holoviews.org/reference/elements/bokeh/QuadMesh.html) plot from Holoviews. For a more in-depth tutorial for visualizing gridded data in Holoviews, go to [Gridded Datasets](http://holoviews.org/getting_started/Gridded_Datasets.html).

In [ ]:
prec_time.hvplot.quadmesh(x='lon', y='lat', rasterize=True, geo=True, tiles='OSM', alpha=0.7, cmap='turbo')

### We can also look at a time-series for a specific grid cell

In [ ]:
prec_point = prec.isel(y=600,x=600).sel(time=slice('2015-02-11 00:00','2015-04-28 00:00')).load()

Note the previous cell uses the .isel() method, which returns the dataset from where the **x** and **y** indexes equal 600 prior to filtering by **time** and loading the data into memory.

Lets plot the dataset.

In [ ]:
prec_point.hvplot(x='time', grid=True)

## Importing geographic extents
Sometimes the data that is brought in is not analyzed but is used to clip a larger dataset to an area of interest (AOI).  <br>
Let's look at two ways this can be done: a user-defined polygon or using the pyNHD package. Data can also be brought in other ways such as a local file or an API request. These are covered in other tutorials. <br>
We'll show how to use geometries to limit datasets later in this notebook.

### The first method will use the the Holoviews and Geoviews libraries to let a user draw and then add the polygon dimensions to a geopandas GeoDataFrame.

When the next code block is run, a map will open and the PolyDraw tool automatically selected. Double tap to add the first vertex, then use tap to add each subsequent vertex, to finalize the draw action double tap to insert the final vertex or press the ESC key to stop drawing.

In [ ]:
#use CartoLight basemap
basemap = gv.tile_sources.CartoLight()

# x and y limits for CONUS
xlim = (-135, -50)
ylim = (22, 50)

#create blank polygon to draw
## redim.range works with Bokeh backend to set default map extent
blank_poly = gv.Polygons([]).redim.range(Longitude=xlim, Latitude=ylim)

# set PolyDraw for creation and PolyEdit for editing polygon, num_objects keeps to single object at a time
user_poly = PolyDraw(source=blank_poly, show_vertices=True, num_objects=1)
user_poly_edit = PolyEdit(source=blank_poly)

# create plots
## active_tools set to allow instant polygon drawing
basemap.options(width=700, height=400) * blank_poly.options(
    active_tools=['poly_draw'], fill_alpha=0.2, line_color='black')

The next code block pulls the latitude and longitude coordinates for the polygon vertices that were just drawn and creates a polygon GeoDataFrame.

In [ ]:
# extract lists of lat/long coordinates
long = user_poly.data['xs'][0]
lat = user_poly.data['ys'][0]

# create zip of polygon vertices
vertices = zip(long, lat)

# construct polygon in GDF
polygon = gpd.GeoDataFrame(
    index=[0], crs="EPSG:4326", geometry=[Polygon(vertices)])

In [ ]:
# plot polygon to confirm the shape matches what was drawn
polygon.plot()

### The second method will be importing the HUC6 boundaries using the PyGeoHydro library. PyGeoHydro is a part of the HyRiver library and is document [here](https://docs.hyriver.io/autoapi/pygeohydro/index.html).

The following cell queries the Water Boundary Dataset HUC6 layer and returns a GeoDataFrame from the .byids() function by examing the "huc6" field for the list of HUC6 id's. 

In [ ]:
drb = pygeohydro.WBD("huc6", outfields=["huc6", "name"]).byids("huc6", ["020401", "020402"])
drb

You see there are two polygons in the GeoDataFrame and ploting it them confirms this.

In [ ]:
drb.plot()

When you want to use geometries to refine datasets to an AOI, it is best to have a single, concise geometry. We'll combine them in the next code cell.

In [ ]:
# create a column where all entries have the same value
drb["name"] = "DRB"

#dissolve by that column
drb = drb.dissolve(by="name")

Check to make sure it worked by examing the tabular and spatial data.

In [ ]:
# tabular 
drb

In [ ]:
# spatial
drb.plot()

## **Putting it together: Preprocess CONUS404 to variable and research spatial extent**
In this section we are going to put together some skills we have learned so far: bring in CONUS404, select our variables and time extent, then clip to our spatial extent.

Variables: Accumulated precipitation (PREC_ACC_NC), air temperature (TK), and surface net radiation (RNET) <br>
Time period: 01/01/1990 - 12/31/1999 <br>
Spatial extent: Delaware River Basin <br>

In [5]:
# set up conus404 filename
conus404 = 'conus404-hourly-cloud'

# create dask array from dataset
ds = cat[conus404].to_dask()
ds = ds.metpy.parse_cf()

Get CRS from CONUS404 dataset 

In [6]:
crs = ds.TK.metpy.cartopy_crs
# crs

# cell below caches

In [7]:
# bring in boundaries of DRB and create single polygon
drb = pygeohydro.WBD("huc6", outfields=["huc6", "name"]).byids("huc6", ["020401", "020402"])
# create a column where all entries have the same value
drb["name"] = "DRB"

# dissolve by that column
drb = drb.dissolve(by="name")

# set CRS to match ds
drb = drb.iloc[[0]].to_crs(crs)

#visualize
# drb.plot()

In [8]:
#subset data variables
c404_variables = ["PREC_ACC_NC", "TK", "ACSWDNB", "I_ACSWDNB", "ACSWDNB", "I_ACSWDNB", "ACLWDNB", "I_ACLWDNB", "ACLWUPB", "I_ACLWUPB"]
c404 = ds[c404_variables]

# write CRS
c404.rio.write_crs(crs, inplace=True)

# perform clip
c404_drb = c404.rio.clip(drb.geometry, crs=crs, drop=True, invert=False)

Visualize the results

In [ ]:
c404_drb["ACSWDNB"].isel(time=-1).hvplot(x='x', y='y', crs=crs, rasterize=True, cmap='turbo', tiles='OSM')

We have a little more processing to do before the dataset is ready for analysis. We need to:
1. Calcuate RNET using the radiation columns
2. Resample and aggregate the data to the desired time-step (30 days)

RNET is calculated using the equation <br>

Lets break down the components. First, lets tackle SWDN, which is calculated like this: <br>
(ACSWDNB[i]+(1e9xI_ACSWDNB[i])) - (ACSWDNB[i-1]+(1e9xI_ACSWDNB[i-1])) / 3600

In [ ]:
"ACSWDNB", "I_ACSWDNB", "ACSWDNB", "I_ACSWDNB", "ACLWDNB", "I_ACLWDNB", "ACLWUPB", "I_ACLWUPB"

We want all values for ACSWDNB and I_ACSWDNB starting at index = 1....

In [9]:
ACSWDNB = c404_drb["ACSWDNB"][1:]

I_ACSWDNB = c404_drb["I_ACSWDNB"][1:]

and we want all values for ACSWDNB before index = -1 and have the same time index the other array.

In [10]:
ACSWDNB1 = c404_drb["ACSWDNB"][:-1]
ACSWDNB1.coords["time"] = ACSWDNB.coords["time"]

I_ACSWDNB1 = c404_drb["I_ACSWDNB"][:-1]
I_ACSWDNB1.coords["time"] = I_ACSWDNB.coords["time"]

Confirm both time coords are the same length.

In [ ]:
len(ACSWDNB.coords["time"].values) == len(ACSWDNB1.coords["time"].values)

Multiply the *I_* arrays by 1e9

In [11]:
I_ACSWDNB_1e9 = 1e9*I_ACSWDNB
I_ACSWDNB1_1e9 = 1e9*I_ACSWDNB1

Now calculate SWDN

In [54]:
SWDN = ((ACSWDNB + I_ACSWDNB_1e9) - (ACSWDNB1 + I_ACSWDNB1_1e9)) / 3600

Let's visualize some of the results.

In [15]:
SWDN.sel(time="2000-06-01 10:00").hvplot(x='x', y='y', crs=crs, rasterize=True, cmap='turbo', tiles='OSM')

:DynamicMap   []
   :Overlay
      .Tiles.I :Tiles   [x,y]
      .Image.I :Image   [x,y]   (value)

In [16]:
SWDN.sel(time="2000-06-01 23:00").hvplot(x='x', y='y', crs=crs, rasterize=True, cmap='turbo', tiles='OSM')

:DynamicMap   []
   :Overlay
      .Tiles.I :Tiles   [x,y]
      .Image.I :Image   [x,y]   (value)

Next, pad a NaN to the beginning to match original datasets dimension length and then reset to those dimensions

In [55]:
SWDN = SWDN.pad({"time":(1,0)})
SWDN.coords["time"] = c404_drb["ACSWDNB"].coords["time"]

### Preprocess reference data

Now that the CONUS404 dataset has been preprocessed, it is also import for analysis to do the same with the reference data used in the comparitive analysis. In this section, data will be brought in from several sources and preprocessed in data type appropriate ways.

#### NOAA's Global Historical Climate Network - Daily (GHCN) Dataset
It is always important to review and readme or metadata files for the data you wish to bring in. The [GHCN readme](https://noaa-ghcn-pds.s3.amazonaws.com/readme.txt) is useful because it explains what is in the S3 bucket, the various columns in the datasets, and other information. When we later call in the observational data, the [by station readme](https://noaa-ghcn-pds.s3.amazonaws.com/readme-by_station.txt) provides a more detailed explanation of the data there.

After reading the metadata for the file, it can be seen that only the first three columns are needed to map the stations: the station ID, latitude, and longitude. However, we want to make sure that we are only using HCN stations so we need to also use the HCN/CRN Flag column to filter to HCN sites. 

Start by getting a list of stations from the AWS S3 bucket where the daily data is housed.

In [ ]:
ghcn_all = pd.read_csv('s3://noaa-ghcn-pds/ghcnd-stations.txt', sep="\t", header=None)
# ghcn_all.head(2)

As you can see, the file reads in as all the contents from one line ending up in one column. 

So we have to split the column into other columns and retain only the needed columns.

In [ ]:
ghcn_all = ghcn_all[0].str.split(" +",expand = True)
# ghcn_all.head(2)

As you can see, columns 0-3 look as we'd expect. However, column 4 is where it starts to get messy as the method for expanding the columns has split up the station names at the spaces between. This means that the HCN flag, which we would expect to be in column 6, could be in columns 6-13. Thankfully, the pandas ```loc``` function makes do this filtering easy.

In [ ]:
ushcn = ghcn_all.loc[(ghcn_all[6] == "HCN") | (ghcn_all[7] == "HCN") | (ghcn_all[8] == "HCN") | (ghcn_all[9] == "HCN") | (ghcn_all[10] == "HCN") | (ghcn_all[11] == "HCN") | (ghcn_all[12] == "HCN") | (ghcn_all[13] == "HCN")].copy()
ushcn = ushcn.iloc[:, 0:3].rename({0:"station", 1:"lat", 2:"lon"}, axis=1).copy() # after the search, trim the columns and rename to get the data to what is needed to map
# ushcn.head()

We now need to clip the points to only those in the DRB. We do that by using the latitude and longitude to create a GeoDataFrame...

In [ ]:
ushcn_gdf = gpd.GeoDataFrame(ushcn, geometry=gpd.points_from_xy(ushcn['lon'], ushcn['lat'], crs="EPSG:4326"))

# convert to same crs as drb
ushcn_gdf = ushcn_gdf.to_crs(crs)

# ushcn_gdf.plot()

followed by clipping using the *drb* geodataframe above

In [ ]:
drb_hcn = gpd.clip(ushcn_gdf, drb)
# drb_hcn.plot()

Now we want to pull in the tabular data for all of the DRB stations. These are stored on AWS in an individual CSV for each station named *station.csv*. So, we need to get all of the station IDs from our dataset and use them to create a list of URLs for these.

In [ ]:
drb_hcn_data_url = [f"s3://noaa-ghcn-pds/csv/by_station/{station}.csv" for station in drb_hcn["station"].unique().tolist()]
# print(drb_hcn_data_url[0:3])

In [ ]:
len(drb_hcn_data_url)

We'll now pass that list of URLs to *dask.dataframe.read_csv*, which will read the data in parallel. We'll then refine the entries to those in 1979 and after.

In [ ]:
drb_hcn_data = dd.read_csv(drb_hcn_data_url, parse_dates=["DATE"], usecols=["ID", "DATE", "ELEMENT", "DATA_VALUE"])

drb_hcn_data = drb_hcn_data.loc[drb_hcn_data["DATE"] >= "1979-01-01"]

Next, we'll refine the dataframe by a list of elements and then compute it.

##### Note: We are using TMAX and TMIN rather than TAVG as TAVG has no records prior to 1998.

In [ ]:
# list of elements we are interested in
element_list = ["PRCP", "TMAX", "TMIN"]

drb_hcn_data = drb_hcn_data.loc[drb_hcn_data["ELEMENT"].isin(element_list)]

In [ ]:
# check shape
drb_hcn_data.compute().shape

In [ ]:
# how much memory does it take up?
drb_hcn_data.compute().memory_usage()

The dask dataframe is about 21 mb in size.

Similar to the CONUS404 data, we have a little more engineering to do with the data. We need to calculate the average temperatue using TMIN and TMAX (in Kelvin) as well as resample the data to a 1 month interval. We'll convert the Dask Dataframe into a Pandas Dataframe to do this.

In [ ]:
drb_hcn_data_df = drb_hcn_data.compute()

We start by whittling down to our two elements, dropping the *ELEMENT* column, and grouping our data by *ID* and *DATE* in order to take the mean of *TMIN* and *TMAX* and convert this to degrees Kelvin.

In [ ]:
# paring down data
drb_hcn_tk = drb_hcn_data_df.loc[drb_hcn_data_df["ELEMENT"].isin(["TMAX","TMIN"])]

# dropping ELEMENT
drb_hcn_tk = drb_hcn_tk.drop("ELEMENT", axis=1)

# calculate mean temperature for each station and date
drb_hcn_tk = drb_hcn_tk.groupby(["ID", "DATE"]).mean()

# rename the DATA_VALUE column to TK
drb_hcn_tk.rename({"DATA_VALUE":"TK"}, axis=1)

# convert from tenths of degrees Celsius to degrees Kelvin
drb_hcn_tk["TK"] = (drb_hcn_tk["TK"] * 0.1) + 273.15

# reset the index
drb_hcn_tk.reset_index(inplace=True)

#### NOAA's Global Climate Reference Network (GCRN) Dataset

Use *fsspec* to make FTP call to NOAA for CRN data <br>
First, create file system

In [ ]:
fs = FTPFileSystem("ftp.ncei.noaa.gov")

Since the file type is a *tsv*, we will use the *pd.read_table* function to create a Dataframe

In [ ]:
crn_stations = pd.read_table(fs.open("/pub/data/uscrn/products/stations.tsv")) 
crn_stations.head()

Now turn into GDF

In [ ]:
crn_stat = gpd.GeoDataFrame(crn_stations, geometry=gpd.points_from_xy(crn_stations["LONGITUDE"], crn_stations["LATITUDE"]), crs="EPSG:4326")

# convert to same crs as drb
crn_stat = crn_stat.to_crs(crs)

crn_stat.plot()

Find which USCRN sites are in DRB

In [ ]:
drb_crn = gpd.clip(crn_stat, drb)
drb_crn.plot()

In [ ]:
drb_crn.head()

We now know what CRN sites are in the Delaware River Basin. We must now retrieve the data for this site from the FTP server.

First, we'll get the location name.

In [ ]:
crn_stat_name = drb_crn["LOCATION"].values.tolist()[0]
print(crn_stat_name)

Initilize the FTP connection again

In [ ]:
fs = FTPFileSystem("ftp.ncei.noaa.gov")

In [ ]:
file_list_glob = fs.glob(
    f"/pub/data/uscrn/products/daily01/**/*{crn_stat_name}*")

In [ ]:
crn_data = pd.DataFrame()

for file in file_list_glob:
    stat_data = pd.read_csv(fs.open(file), header=None, sep="\t")
    crn_data = pd.concat([crn_data, stat_data])

In [ ]:
crn_data = crn_data[0].str.split(" +",expand = True)
crn_data.head()

Now bring in the headers for the station data

In [ ]:
crn_headers = fs.open("/pub/data/uscrn/products/daily01/headers.txt")
crn_data_headers = pd.read_csv(crn_headers, sep="\t", header=None).iloc[1,:].str.split(" +").values.tolist()[0][0:28]
crn_data_headers

Check that the number of headers equals the number of columns in our data

In [ ]:
print(len(crn_data.columns) == len(crn_data_headers))

Now then rename the column headers

In [ ]:
crn_data.columns = crn_data_headers
crn_data

Convert the *LST_DATE* column to datetime and refine columns to those of interest.

In [ ]:
crn_data["DATE"] = pd.to_datetime(crn_data["LST_DATE"])
crn_data = crn_data[["DATE", "P_DAILY_CALC", "T_DAILY_MAX", "T_DAILY_MIN", "SOLARAD_DAILY"]]

In [ ]:
crn_data.head()

If you examine the data types, you'll see that the 4 columns of numbers are actually data type *object*

In [ ]:
crn_data.dtypes

Lets rectify that by applying the `pd.to_numeric` function to the columns.

In [ ]:
cols = crn_data.columns.drop("DATE")
crn_data[cols] = crn_data[cols].apply(pd.to_numeric, errors='coerce')
crn_data.dtypes

#### PRISM data
This time we will open the PRISM dataset, clip it, and refine the data. Many of the steps will look the same as the CONUS404 dataset so there will be less explanation of the steps.

In [ ]:
fs = fsspec.filesystem("s3", anon=False, requester_pays=True, skip_instance_cache=True)

In [ ]:
prism_years = range(1970, 2021, 1)
chunks={"time": 6, "lon": 703, "lat": 311}
pr = [xr.open_dataset(fs.open(f"s3://nhgf-development/thredds/prism_v2/prism_{str(year)}.nc"), chunks=chunks, decode_coords="all") for year in prism_years]
prism = xr.concat(pr, dim="time")
prism = prism.drop_vars("time_bnds")
# prism

## Testing using AlbersEqualArea to write crs for prism so it displays properly after transformation

In [ ]:
prism_crs = ccrs.AlbersEqualArea()

In [ ]:
prism.rio.write_crs(4269, inplace=True)

Rename the dimensions to match cd conventions used by rioxarray

In [ ]:
prism = prism.rename({"lon":"x", "lat":"y"})
# prism

In [ ]:
prism.hvplot(x="x", y="y", geo=True, rasterize=True, tiles="OSM")

Reproject from NAD83 to Lambert Conformal Conic (matches CONUS404 dataset)

In [ ]:
prism_reproject = prism.rio.reproject(crs, num_threads=30)

In [ ]:
prism_reproject

In [ ]:
import numpy as np
xmin, ymin = prism_reproject.x[0].values, prism_reproject.y[0].values

y = ymin - np.arange(872)*4000.

x = xmin + np.arange(1602)*4000.

prism_reproject = prism_reproject.assign_coords(x=x, y=y)

In [ ]:
prism_reproject

In [ ]:
prism_reproject.hvplot(x="x", y="y", rasterize=True,,tiles="OSM")

In [ ]:
# clip to DRB
prism_drb = prism_reproject.rio.clip(drb.geometry, crs=crs, drop=True, invert=False)

Display the clipped data

In [ ]:
prism_drb.hvplot.quadmesh(x='x', y='y', rasterize=True, tiles='OSM', alpha=0.7, cmap='turbo')

In [ ]:
# del prism_reproject
# del prism_drb

In [ ]:
# # Last code cell of the notebook
# import watermark.watermark as watermark
# print(watermark(iversions=True, python=True, machine=True, globals_=globals()))

In [ ]:
client.close(); cluster.shutdown()